In [ ]:
# Importing necessary libraries
import pandas as pd
import json
import fitz     
import httpx

url = "http://localhost:11434/api/generate"

In [ ]:
# File paths
pdf_files = {
    "signalment_physical": "signalment_physical.pdf",
    "cbc": "cbc.pdf",
    "chem": "chem.pdf",
    "cpli": "cpli.pdf",
    "aus": "aus.pdf"
}

csv_file = "table.csv"

In [64]:
# Function to extract the text from file
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

pdf_texts = {name: extract_text_from_pdf(path) for name, path in pdf_files.items()}

# Load the CSV file
df = pd.read_csv(csv_file, usecols=[0, 1])    # Only reading the first two columns, as the other two may have values which will be overwritten
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # One item has extra spaces, so I am stripping any trailing whitespaces

In [65]:
df["details"] = ""

# Specifying rows that require details as rows 6-12 and 55-60
df.loc[4:10, "details"] = "details"
df.loc[53:58, "details"] = "details"

In [66]:
pdf_texts

{'cbc': 'Lab No:\nSpecimen:\nSample:\nAttending Clinician:\nDate Submitted:\nDate Finalized:\nPertinent History:\nEDTA\nblood\nDKA\nSenior Clinician:\nClinical Service:\nPriority:\nSample Collected:\nSample Received:\nFasting Sample:\nZoonotic Concern:\nNo\nEmail Results:\nNo/No\nFollow up Tests:\nNo\nTest Name\nResult\nFlag\nReference\nUnit\nWBC\n28.5\nH\n6.0 - 17.0\nul\nRed Blood Cell Count\n4.64\nL\n5.50 - 8.50\nM/ul\nHemoglobin\n9.9\nL\n10.0 - 20.0\ng/dl\nPacked Cell Volume\n29.8\nL\n31.0 - 56.0\n%\nMean Corpuscular Volume\n64.2\n60.0 - 77.0\nfl.\nMean Corpuscular Hemoglobin \nConcentration\n33.2\n32.0 - 36.0\ng/dl\nPlasma Protein\n6.5\n6 - 8\nTS-g/dl\nFibrinogen (heat precipitation)\nmg/dl\nPlatelet Count (Automated)\n347000\n200000 - 500000\n/ul\nSegmented Neutrophils\n86\nH\n60 - 77\n%\nAbsolute Neutrophil\n24510\nH\n3000 - 11500\nBand\n0 - 3\n%\nAbsolute Bands\n0 - 300\nMetamyelocyte\n%\nAbsolute Metamylocyte\nLymphocytes\n4\nL\n12 - 30\n%\nAbsolute Lymphocyte\n1140\n1000 - 480

In [67]:
# Merge all pdf text -> Increased memory usage and CPU time
# Shorter prompts, more API calls -> Increased CPU load
# Parallel API calls -> Highest CPU consumption
# Could use batching?
# I need a compatible GPU

# Models in consideration:
# mistral (best but heavy), mistral:7b-instruct-q4_0 (4-bit quantized)
# tinyllama (testing)

def extract_with_ollama(prompt): 
    
    data = {
        "model": "mistral",
        "prompt": prompt
    }

    response = httpx.post(url, data=json.dumps(data), headers={'Content-Type': 'application/json'}, timeout=300)
    response_lines = [line for line in response.text.strip().split('\n') if line]
    response_dicts = [json.loads(line) for line in response_lines]
    result = ''.join(response_dict.get('response', '') for response_dict in response_dicts)
    return result

In [ ]:
def create_df(type, text, items):
    if type == "results":

        prompt = f"""
            Extract the value, including units if any, for the following item/items from the veterinary medical report. Return only the value in a valid JSON format and nothing else.

            Item/Items: {items}

            Report Text:
            {text}

            The output should be in the following JSON structure:
            {{
                "neuter status": "yes",
                "height": "12 inch"
            }}

            Conditions:
            - Only the specified item should be returned.
            - If the item is not found, return "N/A".
            - Do not add any extra text, comments, conclusions or explanations.
            - Ensure the JSON format is valid with no extra formatting.
        """

    else:
        prompt = f"""
            Extract the exact text related to the following item/items from the veterinary medical report and return them in valid JSON format.
            
            ### **Item/Items:**  
            {items}
            
            ### **Report Text:**  
            {text}

            ### **Extraction Instructions:**
            - Extract **only the specific portion of text** that directly corresponds to each item.
            - **Do NOT include extra context** or unrelated symptoms. Each extracted text must be as **precise** as possible.
            - If an item is **not found**, return "N/A".
            - The JSON format must follow this structure, ensuring only the **most relevant** text is extracted:

            ### **Output Format (JSON):**
            {{
                "item": "exact relevant text"
            }}
        """
        
    str_data = extract_with_ollama(prompt)
    data = json.loads(str_data)
    df = pd.DataFrame(list(data.items()), columns=['items', type])
    return df

In [69]:
final_df = pd.DataFrame()

for pdf_name, text in pdf_texts.items():

    # Get results values from text
    items = ", ".join(df.loc[df["filename"] == pdf_name, "items"].astype(str))
    results_df = create_df("results", text, items)

    # Get details from text
    if pdf_name in {"signalment_physical", "aus"}:
        items = ", ".join(df.loc[(df["filename"] == pdf_name) & (df["details"].ne("")), "items"].astype(str))
        details_df = create_df("details", text, items)
        merged_df = pd.merge(results_df, details_df, on="items", how="left")
    else:
        merged_df = results_df
    
    print(f"{pdf_name} has completed processing")
    final_df = pd.concat([final_df, merged_df], ignore_index=True)

 {
       "wbc": 28.5,
       "red_blood_cell_count": 4.64,
       "hemoglobin": 9.9,
       "packed_cell_volume": 29.8,
       "mean_corpuscular_volume": 64.2,
       "mean_corpuscular_hemoglobin_concentration": 33.2,
       "plasma_protein": 6.5,
       "platelet_count": 347000,
       "absolute_neutrophil": 24510,
       "absolute_bands": 0,
       "absolute_lymphocyte": 1140,
       "absolute_monocyte": 2850,
       "absolute_eosinophil": 100,
       "absolute_basophil": "N/A",
       "absolute_other": "N/A"
     }
cbc has completed processing
 {
       "glucose": "396",
       "lactic_acid": "15.2",
       "blood_urea_nitrogen": "12",
       "creatinine": "0.56",
       "sodium": "132",
       "potassium": "3.5",
       "enzymatic_carbon_dioxide": "9",
       "chloride": "101",
       "anion_gap_calculated": "26.3",
       "calcium": "8.2",
       "phosphorus": "3.2",
       "magnesium": "1.3",
       "total_protein": "5.5",
       "albumin": "2.9",
       "globulin": "2.5",
     

In [70]:
# Save the updated CSV
final_df.to_csv("output_table.csv", index=False)

print("Extraction complete using Ollama.")

Extraction complete using Ollama.


In [71]:
final_df

,items,results
0,wbc,28.5
1,red_blood_cell_count,4.64
2,hemoglobin,9.9
3,packed_cell_volume,29.8
4,mean_corpuscular_volume,64.2
5,mean_corpuscular_hemoglobin_concentration,33.2
6,plasma_protein,6.5
7,platelet_count,347000
8,absolute_neutrophil,24510
9,absolute_bands,0
